In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)

from xlsxwriter import Workbook

In [3]:
# Environmnet files from Google Spreadsheet
envMem_df = pd.read_csv('./sources/envMem.csv')
envCPU_df = pd.read_csv('./sources/envCPU.csv')
envFS_df = pd.read_csv('./sources/envFS.csv')

# Weekly data from Splunk
prodMem_df = pd.read_csv('./sources/prodMem.csv')
stgMem_df = pd.read_csv('./sources/stgMem.csv')
prodCPU_df = pd.read_csv('./sources/prodCPU.csv')
stgCPU_df = pd.read_csv('./sources/stgCPU.csv')
prodFS_df = pd.read_csv('./sources/prodFS.csv')

In [4]:
# To replace the NaN
envMem_df = envMem_df[envMem_df['Host'].isna() == False]
envCPU_df = envCPU_df[envCPU_df['Host'].isna() == False]
#envFS_df = envFS_df[envFS_df['Mount'].isna() == False]

prodMem_df = prodMem_df[prodMem_df['Host'].isna() == False]
stgMem_df = stgMem_df[stgMem_df['Host'].isna() == False]
prodCPU_df = prodCPU_df[prodCPU_df['Host'].isna() == False]
stgCPU_df = stgCPU_df[stgCPU_df['Host'].isna() == False]
prodFS_df = prodFS_df[prodFS_df['Used'].isna() == False]

In [5]:
# Merging .csv files for Memory usage
prod_Mem_mergeData = pd.merge(envMem_df, prodMem_df, left_on='Host', right_on='Host', how='right').sort_values('IP Address_x')
stg_Mem_mergeData = pd.merge(envMem_df, stgMem_df, left_on='Host', right_on='Host', how='right').sort_values('IP Address_x')

# Merging .csv files for CPU usage
prod_CPU_mergeData = pd.merge(envCPU_df, prodCPU_df, left_on='Host', right_on='Host', how='right').sort_values('IP Address_x')
stg_CPU_mergeData = pd.merge(envCPU_df, stgCPU_df, left_on='Host', right_on='Host', how='right').sort_values('IP Address_x')

# Merging the two .csv files according to 'Host' and 'Mount' via inner join for FS usage (PROD only)
prod_FS_mergeData = pd.merge(envFS_df,prodFS_df, left_on=['Host', 'Mount'],right_on=['Host','Mount'], how='inner')
#prod_FS_mergeData[['Host', 'Mount', 'Used']].sort_values(['Host', 'Mount']) # uncomment to print to screen / comment to unprint
mergeDataPrint = prod_FS_mergeData[['Host', 'Mount', 'Used']].sort_values(['Host', 'Mount']) 
mergeDataPrint.head(264).to_csv('./raw/prod_FS_Weekly_Output.csv', index = False) 
print(mergeDataPrint.head(264))

                                                 Host                Mount  Used
0                                        dpaanalytics                    /  15.0
1                                        dpaanalytics          /appl/depot   1.0
2                                        dpaanalytics  /appl/di_shareddata   1.0
3                                        dpaanalytics      /appl/sasbackup   1.0
4                                        dpaanalytics        /appl/sasdata   1.0
..                                                ...                  ...   ...
250  ip-10-237-82-146.ap-southeast-1.compute.internal      /appl/sasbackup   1.0
251  ip-10-237-82-146.ap-southeast-1.compute.internal        /appl/sashome   4.0
252  ip-10-237-82-146.ap-southeast-1.compute.internal        /appl/saslogs   1.0
253  ip-10-237-82-146.ap-southeast-1.compute.internal                 /dev   0.0
254  ip-10-237-82-146.ap-southeast-1.compute.internal                /work  46.0

[255 rows x 3 columns]


In [6]:
# Assigning variables for printing
print_prod_Mem = prod_Mem_mergeData[['Host', 'IP Address_x', 'Used']]
print_stg_Mem = stg_Mem_mergeData[['Host', 'IP Address_x', 'Used']]
print_prod_CPU = prod_CPU_mergeData[['Host', 'IP Address_x', 'Used']]
print_stg_CPU = stg_CPU_mergeData[['Host', 'IP Address_x', 'Used']]

In [7]:
# Uncomment to PRINT TO SCREEN / comment to hide
#print_prod_Mem.head(29)
#print_stg_Mem.head(30)
#print_prod_CPU
#print_stg_CPU.head(30)
#prodFS_df.head(10)
#prodFS_df

In [8]:
# Exporting to .csv for individual utils
print_prod_Mem.head(29).to_csv('./raw/prod_Mem_Weekly_Output.csv') # working
print_stg_Mem.head(30).to_csv('./raw/stg_Mem_Weekly_Output.csv') # working
print_prod_CPU.head(29).to_csv('./raw/prod_CPU_Weekly_Output.csv') # working
print_stg_CPU.head(30).to_csv('./raw/stg_CPU_Weekly_Output.csv') # working

# PROD & STG MEM - Appending the two .csv output for Mem
mem_prod = pd.read_csv('./raw/prod_Mem_Weekly_Output.csv')
mem_stg  = pd.read_csv('./raw/stg_Mem_Weekly_Output.csv')
mem_weekly = mem_prod.append(mem_stg)# <-- append step
mem_weekly = mem_weekly.drop(mem_weekly.columns[[0]], axis=1) # <-- removes additional column created
mem_weekly.reset_index(drop=True, inplace=True) # <-- resets the index number
mem_weekly.to_csv('./raw/mem_weekly.csv', index = False) # <-- index=False to remove index upon saving
#print(mem_weekly)

# PROD & STG CPU - Appending the two .csv output
prod_CPU = pd.read_csv('./raw/prod_CPU_Weekly_Output.csv')
stg_CPU  = pd.read_csv('./raw/stg_CPU_Weekly_Output.csv')
CPU_weekly = prod_CPU.append(stg_CPU) # <-- append step
CPU_weekly = CPU_weekly.drop(CPU_weekly.columns[[0]], axis=1) # <-- removes additional column created
CPU_weekly.reset_index(drop=True, inplace=True) # <-- resets the index number
CPU_weekly.to_csv('./raw/CPU_weekly.csv', index = False) # <-- index=False to remove index upon saving
#print(CPU_weekly)

# Prod FS
prod_weekly = pd.read_csv('./raw/prod_FS_Weekly_Output.csv')

# Display the number of rows and columns (rows,columns)
#mem_weekly.shape
#CPU_weekly.shape
#prod_weekly.shape

# Display the number of rows and columns (rows,columns). Uncomment to view
writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
CPU_weekly.to_excel(writer, sheet_name = 'CPU', index = False)
mem_weekly.to_excel(writer, sheet_name = 'mem', index = False)
prod_weekly.to_excel(writer, sheet_name = 'FS', index = False)
writer.save()